In [4]:
!unzip utkface-new.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
fldr="/content/UTKFace"

In [6]:
import os
files=os.listdir(fldr)
print(int(files[0].split('_')[0]))
print(files[0])

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/UTKFace'

In [ ]:
import cv2;

ages=[]
genders=[]
images=[]

for i, fle in enumerate(files):
  age=int(fle.split('_')[0])
  gender=int(fle.split('_')[1])
  total=fldr+'/'+fle
  image=cv2.imread(total)

  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image= cv2.resize(image,(48,48))
  images.append(image)
#   if i % 1000 == 0:
#     print(i)

In [ ]:
for fle in files:
  age=int(fle.split('_')[0])
  gender=int(fle.split('_')[1])
  ages.append(age)
  genders.append(gender)

In [14]:
# from google.colab.patches import cv2_imshow
# cv2_imshow(images[24])

In [ ]:
print(ages[24])
print(genders[24])

In [16]:
# cv2_imshow(images[53])

In [ ]:
print(ages[53])
print(genders[53])

In [ ]:
import numpy as np
images_f=np.array(images)
genders_f=np.array(genders)
ages_f=np.array(ages)

In [ ]:
np.save('image.npy',images_f)
np.save('gender.npy',genders_f)
np.save('age.npy',ages_f)

Male = 0
Female= 1

In [ ]:
values, counts = np.unique(genders_f, return_counts=True)
print(counts)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
gender = ['Male', 'Female']
values=[4372,5047]
ax.bar(gender,values)
plt.show()

In [ ]:
values, counts = np.unique(ages_f, return_counts=True)
print(counts)

In [ ]:
val=values.tolist()
cnt=counts.tolist()

In [ ]:
plt.plot(counts)
plt.xlabel('ages')
plt.ylabel('distribution')
plt.show()

In [ ]:
labels=[]

i=0
while i<len(ages):
  label=[]
  label.append([ages[i]])
  label.append([genders[i]])
  labels.append(label)
  i+=1

In [ ]:
images_f_2=images_f/255
labels_f=np.array(labels)
images_f_2.shape

In [21]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, Y_train, Y_test= train_test_split(images_f_2, labels_f,test_size=0.25)

In [ ]:
Y_train[0:5]

In [ ]:
Y_train_2=[Y_train[:,1],Y_train[:,0]]
Y_test_2=[Y_test[:,1],Y_test[:,0]]

In [ ]:
Y_train_2[0][0:5]

In [ ]:
Y_train_2[1][0:5]

In [168]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten,BatchNormalization
from tensorflow.keras.layers import Dense, MaxPooling2D,Conv2D
from tensorflow.keras.layers import Input,Activation,Add
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

def Convolution(input_tensor,filters):
    
    x = Conv2D(filters=32,kernel_size=(3, 3),padding = 'same',kernel_regularizer=l2(0.01))(input_tensor)
    x = Dropout(0.2)(x)
    x= Activation('relu')(x)

    return x
def model(input_shape):
  inputs = Input((input_shape))
  
  conv_1= Convolution(inputs,64)
  maxp_1 = MaxPooling2D(pool_size = (2,2)) (conv_1)
  conv_2 = Convolution(maxp_1,32)
  maxp_2 = MaxPooling2D(pool_size = (2, 2)) (conv_2)
  conv_3 = Convolution(maxp_2,64)
  maxp_3 = MaxPooling2D(pool_size = (2, 2)) (conv_3)
  conv_4 = Convolution(maxp_3,512)
  maxp_4 = MaxPooling2D(pool_size = (2, 2)) (conv_4)
  flatten= Flatten() (maxp_4)
  dense_1= Dense(64,activation='relu')(flatten)
  dense_2= Dense(64,activation='relu')(flatten)
  drop_1=Dropout(0.2)(dense_1)
  drop_2=Dropout(0.2)(dense_2)
  output_1= Dense(1,activation="sigmoid",name='sex_out')(drop_1)
  output_2= Dense(1,activation="relu",name='age_out')(drop_2)
  model = Model(inputs=[inputs], outputs=[output_1,output_2])
  model.compile(loss=["binary_crossentropy","mae"], optimizer="Adam",
	metrics=["accuracy"])
  
  return model

In [169]:
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

In [ ]:
fle_s='model_prediction.h5'
checkpointer = ModelCheckpoint(fle_s, monitor='val_loss',verbose=1,save_best_only=True,save_weights_only=False, mode='auto',save_freq='epoch')
Early_stop=tf.keras.callbacks.EarlyStopping(patience=80, monitor='val_loss',restore_best_weights=True),
callback_list=[checkpointer,Early_stop]

In [ ]:
History=Model.fit(X_train,Y_train_2,batch_size=64,validation_data=(X_test,Y_test_2),epochs=160,callbacks=[callback_list])

In [187]:
from tensorflow.keras.models import load_model
Model = load_model('model_prediction_1.h5')

In [ ]:
Model.evaluate(X_test,Y_test_2)

In [189]:
pred=Model.predict(X_test)

In [ ]:
pred[1]

In [ ]:
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.subplots_adjust(top=1.00, bottom=0.0, left=0.0, right=0.95, hspace=0.25,
                        wspace=0.35)

In [ ]:
plt.plot(History.history['sex_out_accuracy'])
plt.plot(History.history['val_sex_out_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.subplots_adjust(top=1.00, bottom=0.0, left=0.0, right=0.95, hspace=0.25,
                        wspace=0.35)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(Y_test_2[1], pred[1])
ax.plot([Y_test_2[1].min(),Y_test_2[1].max()], [Y_test_2[1].min(), Y_test_2[1].max()], 'k--', lw=4)
ax.set_xlabel('Actual Age')
ax.set_ylabel('Predicted Age')
plt.show()

In [194]:
i=0
Pred_l=[]
while(i<len(pred[0])):

  Pred_l.append(int(np.round(pred[0][i])))
  i+=1


In [195]:
from sklearn.metrics import confusion_matrix 

from sklearn.metrics import classification_report 

In [196]:

report=classification_report(Y_test_2[0], Pred_l)

In [ ]:
print(report)

In [198]:

results = confusion_matrix(Y_test_2[0], Pred_l)

In [ ]:
import seaborn as sns

sns.heatmap(results, annot=True)

In [200]:
def test_image(ind,images_f,images_f_2,Model):
#   cv2_imshow(images_f[ind])
  plt.imshow(images_f[ind])
  image_test=images_f_2[ind]
  pred_1=Model.predict(np.array([image_test]))
  #print(pred_1)
  sex_f=['Male','Female']
  age=int(np.round(pred_1[1][0]))
  sex=int(np.round(pred_1[0][0]))
  print("Predicted Age: "+ str(age))
  print("Predicted Sex: "+ sex_f[sex])

age=int(np.round(pred_1[1][0]))-1310
sex=int(np.round(pred_1[0][0]))

In [211]:
img = cv2.imread('/content/emirhan.jpg')
img = cv2.resize(img,(48,48))
img = np.reshape(img,[48,48,3])

In [ ]:
pred_1=Model.predict(np.array([img]))
#print(pred_1)
sex_f=['Male','Female']
print("Predicted Age: "+ str(age))
print("Predicted Sex: "+ sex_f[sex])
from google.colab.patches import cv2_imshow
images = cv2.imread('/content/emirhan.jpg')
images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB)
plt.imshow(images)
plt.show()

In [213]:
img = cv2.imread('/content/emirhan_1.jpg')
img = cv2.resize(img,(48,48))
img = np.reshape(img,[48,48,3])

In [ ]:
pred_1=Model.predict(np.array([img]))
#print(pred_1)
sex_f=['Male','Female']
print("Predicted Age: "+ str(age))
print("Predicted Sex: "+ sex_f[sex])
from google.colab.patches import cv2_imshow
images = cv2.imread('/content/emirhan_1.jpg')
images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB)
plt.imshow(images)
plt.show()

In [1]:
test_image(57,images_f,images_f_2,Model)

NameError: name 'test_image' is not defined

In [ ]:
test_image(137,images_f,images_f_2,Model)

In [ ]:
test_image(502,images_f,images_f_2,Model)

In [ ]:
test_image(24,images_f,images_f_2,Model)